### Importare le seguenti librerie per eseguire correttamente questo notebook:

- using Pkg
- Pkg.add("CUDA")
- Pkg.add("BenchmarkTools")
- Pkg.add(url="https://github.com/lauriluca99/TGW-3D")
- Pkg.add(url="https://github.com/cvdlab/LinearAlgebraicRepresentation.jl/")


In [87]:
using CUDA
using CUDA.CUSPARSE
using BenchmarkTools
using TGW3D
using NearestNeighbors
using SparseArrays
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation


const Points = Matrix
const ChainOp = SparseArrays.SparseMatrixCSC{Int8,Int}

SparseMatrixCSC{Int8, Int64}

In [2]:
function frag_face(V::Points, EV::ChainOp, FE::ChainOp, 
    sp_idx::Vector{Vector{Int64}}, sigma::Int64)

    vs_num::Int64 = size(V, 1)
    
	# 2D transformation of sigma face
    sigmavs::Vector{Int64} = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV::Points = V[sigmavs, :]
    sEV::ChainOp = EV[FE[sigma, :].nzind, sigmavs]
    M::Points = Lar.Arrangement.submanifold_mapping(sV)
    @views tV::Points = ([V ones(vs_num)]*M)[:, 1:3]
    sV = tV[sigmavs, :]
    
    # sigma face intersection with faces in sp_idx[sigma]
    CUDA.@async for i in sp_idx[sigma]
        tmpV::Points, tmpEV::ChainOp = Lar.Arrangement.face_int(tV, EV, FE[i, :])
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    @views sV = sV[:, 1:2]
    nV::Points, nEV::ChainOp, nFE::ChainOp = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    
    nvsize::Int64 = size(nV, 1)
    @views nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3]
    
    return nV, nEV, nFE
end

frag_face (generic function with 1 method)

In [3]:
function filter_fn(face)
    visited = []
    verts = []
    map(e->verts = union(verts, collect(e)), face)
    verts = Set(verts)
 
    if !(verts in visited)
        push!(visited, verts)
        return true
    end
    return false
end

filter_fn (generic function with 1 method)

In [4]:
function merge_vertices(V::Points, EV::ChainOp, FE::ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)

    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)

    # remove vertices congruent to a single representative
    todelete = []
    i = 1

    for vi in 1:vertsnum #questo for non può essere parallelizzato
        if !(vi in todelete)
            nearvs = Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs = setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            i = i + 1
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]
    
    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)

    for ei in 1:edgenum #questo for non può essere parallelizzato
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end

    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))
 
    etuple2idx = Dict{Tuple{Int, Int}, Int}()

    for ei in 1:nedgenum #questo ciclo non può essere parallelizzato
        begin
            nEV[ei, collect(nedges[ei])] .= 1
            nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    
    #questo ciclo può essere parallelizzato ma non cambia il tempo per un numero di edge piccolo
    @async for e in 1:nedgenum 
        v1,v2 = findnz(nEV[e,:])[1]
        nEV[e,v1] = -1; nEV[e,v2] = 1
    end
 
    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]
 
    @inbounds nfaces = filter(filter_fn, faces)
 
    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))
 
    @async for fi in 1:nfacenum
        @async for edge in nfaces[fi]
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end
 
    return Points(nV), nEV, nFE
 end

merge_vertices (generic function with 2 methods)

In [5]:
function removeinnerloops(g::Int64, nFE::ChainOp)
	FE::Vector{Vector{Int64}} = Lar.cop2lar(nFE)
	nFE::ChainOp = Lar.lar2cop(FE[1:end-g])
	return nFE
end

removeinnerloops (generic function with 1 method)

In [6]:
function spatial_arrangement_1(
    V::Points,
    copEV::ChainOp,
    copFE::ChainOp, multiproc::Bool=false)

# spaceindex computation
FV = Lar.compute_FV( copEV, copFE )
model = (convert(Points,V'), FV)
sp_idx = Lar.spaceindex(model)

# initializations
fs_num = size(copFE, 1)
rV = Array{Float64,2}(undef,0,3)
rEV = SparseArrays.spzeros(Int8,0,0)
rFE = SparseArrays.spzeros(Int8,0,0)
# sequential (iterative) processing of face fragmentation
     for sigma in 1:fs_num
        #print(sigma, "/", fs_num, "\r")
        nV, nEV, nFE = frag_face(V, copEV, copFE, sp_idx, sigma)
        #nV, nEV, nFE = Lar.fragface(V, copEV, copFE, sp_idx, sigma)
        nV = convert(Points, nV)
        rV, rEV, rFE = Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
        #rV=a;  rEV=b;  rFE=c
    end
# merging of close vertices, edges and faces (3D congruence)
rV, rEV, rFE = merge_vertices(rV, rEV, rFE)
return rV, rEV, rFE
end

spatial_arrangement_1 (generic function with 2 methods)

In [7]:
function face_angle(e::Int, f::Int)

    edge_vs = EV[e, :].nzind

    t = findfirst(x->edge_vs[1] in x && edge_vs[2] in x, triangulated_faces[f])

    v1 = normalize(V[edge_vs[2], :] - V[edge_vs[1], :])

    if abs(v1[1]) > abs(v1[2])
        invlen = 1. / sqrt(v1[1]*v1[1] + v1[3]*v1[3])
        v2 = [-v1[3]*invlen, 0, v1[1]*invlen]
    else
        invlen = 1. / sqrt(v1[2]*v1[2] + v1[3]*v1[3])
        v2 = [0, -v1[3]*invlen, v1[2]*invlen]
    end

    v3 = cross(v1, v2)

    M = reshape([v1; v2; v3], 3, 3)

    triangle = triangulated_faces[f][t]
    third_v = setdiff(triangle, edge_vs)[1]
    vs = V[[edge_vs..., third_v], :]*M

    v = vs[3, :] - vs[1, :]
    angle = atan(v[2], v[3])
    return angle
end

face_angle (generic function with 1 method)

In [8]:
function minimal_3cycles(V::Points, EV::ChainOp, FE::ChainOp)

	triangulated_faces = Array{Any, 1}(undef, FE.m)

    #EF = FE'
    EF::ChainOp = convert(ChainOp, LinearAlgebra.transpose(FE))
	FC::ChainOp = Lar.Arrangement.minimal_cycles(face_angle, true)(V, EF)

	#FC'
    return -convert(ChainOp, LinearAlgebra.transpose(FC))
end

minimal_3cycles (generic function with 1 method)

In [9]:
function spatial_arrangement_2(
    rV::Points,
    rcopEV::ChainOp,
    rcopFE::ChainOp, 
    multiproc::Bool=false)

    rcopCF::ChainOp = minimal_3cycles(rV, rcopEV, rcopFE)

    return rV, rcopEV, rcopFE, rcopCF
end

spatial_arrangement_2 (generic function with 2 methods)

In [10]:
function spatial_arrangement_CUDA(
    V::Points, # by rows
    copEV::ChainOp,
    copFE::ChainOp, 
    multiproc::Bool=false)

    # face subdivision
    rV::Points, rcopEV::ChainOp, rcopFE::ChainOp = spatial_arrangement_1( V,copEV,copFE,multiproc)

    rV, rEV::ChainOp, rFE::ChainOp, rCF::ChainOp = spatial_arrangement_2(rV, rcopEV, rcopFE)
end

spatial_arrangement_CUDA (generic function with 2 methods)

# Testing performance

In [101]:
V = [-0.5 -0.5 -0.5 -0.5 0.5 0.5 0.5 0.5 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0; 
    -0.5 -0.5 0.5 0.5 -0.5 -0.5 0.5 0.5 0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0; 
    -0.5 0.5 -0.5 0.5 -0.5 0.5 -0.5 0.5 0.0 1.0 0.0 1.0 0.0 1.0 0.0 1.0]
CV = [[1, 2, 3, 4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14, 15, 16]]
FV = [[1, 2, 3, 4], [5, 6, 7, 8], [1, 2, 5, 6], [3, 4, 7, 8], [1, 3, 5, 7], [2, 4, 6, 8], [9, 10, 11, 12], [13, 14, 15, 16], [9, 10, 13, 14], [11, 12, 15, 16], [9, 11, 13, 15], [10, 12, 14, 16]]
EV = [[1, 2], [3, 4], [5, 6], [7, 8], [1, 3], [2, 4], [5, 7], [6, 8], [1, 5], [2, 6], [3, 7], [4, 8], [9, 10], [11, 12], [13, 14], [15, 16], [9, 11], [10, 12], [13, 15], [14, 16], [9, 13], [10, 14], [11, 15], [12, 16]]

cop_EV = Lar.coboundary_0(EV::Lar.Cells)
cop_FE = Lar.coboundary_1(V, FV::Lar.Cells, EV::Lar.Cells)
W = convert(TGW3D.Points, V')

16×3 Matrix{Float64}:
 -0.5  -0.5  -0.5
 -0.5  -0.5   0.5
 -0.5   0.5  -0.5
 -0.5   0.5   0.5
  0.5  -0.5  -0.5
  0.5  -0.5   0.5
  0.5   0.5  -0.5
  0.5   0.5   0.5
  0.0   0.0   0.0
  0.0   0.0   1.0
  0.0   1.0   0.0
  0.0   1.0   1.0
  1.0   0.0   0.0
  1.0   0.0   1.0
  1.0   1.0   0.0
  1.0   1.0   1.0

In [102]:
#@benchmark Lar.spatial_arrangement(W, cop_EV, cop_FE)

In [103]:
#@benchmark TGW3D.spatial_arrangement(W, cop_EV, cop_FE)

BenchmarkTools.Trial: 535 samples with 1 evaluation.
 Range (min … max):  7.552 ms … 20.287 ms  ┊ GC (min … max): 0.00% … 48.94%
 Time  (median):     8.832 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.342 ms ±  2.494 ms  ┊ GC (mean ± σ):  6.80% ± 12.73%

  ▃▆▇▇▆▆█▇▇▄                                                  
  ██████████▇▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▄▆█▇▅▁▁▄▄▄▁▁▁▆▆▇▆ █
  7.55 ms      Histogram: log(frequency) by time     19.7 ms <

 Memory estimate: 3.14 MiB, allocs estimate: 56899.

In [104]:
@benchmark spatial_arrangement_CUDA(W, cop_EV, cop_FE)

BenchmarkTools.Trial: 531 samples with 1 evaluation.
 Range (min … max):  7.613 ms … 20.800 ms  ┊ GC (min … max): 0.00% … 49.08%
 Time  (median):     8.780 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.418 ms ±  2.594 ms  ┊ GC (mean ± σ):  7.04% ± 12.96%

    █▅ ▁ ▅▄                                                   
  ▃████████▄▃▂▂▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▁▂▃▂▂▂▁▁▁▂▁▂▂▂▃ ▃
  7.61 ms        Histogram: frequency by time          20 ms <

 Memory estimate: 3.19 MiB, allocs estimate: 57614.

The CuArray type is an essential part of the toolchain. Primarily, it is used to manage GPU memory, and copy data from and back to the CPU.

A typical approach for porting or developing an application for the GPU is as follows:

- develop an application using generic array functionality, and test it on the CPU with the Array type
- port your application to the GPU by switching to the CuArray type
- disallow the CPU fallback ("scalar indexing") to find operations that are not implemented for or incompatible with GPU execution
- (optional) use lower-level, CUDA-specific interfaces to implement missing functionality or optimize performance

High-level interface for executing code on a GPU. The @cuda macro should prefix a call, with func a callable function or object that should return nothing.

The type of a texel, which is restricted to the basic integer and single-precision floating-point types and any of the 1-, 2-, and 4-component vector types defined in Built-in Vector Types 2 that are derived from the basic integer and single-precision floating-point types.
https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#texture-memory

In [97]:
#La matrice W ha valori di tipo Float64 che non sono supportati da CuArray
CuArray([W_d])

LoadError: CuArray only supports element types that are stored inline

In [95]:
W_d = CuSparseMatrixCSC(W)
cop_EV_d = CuSparseMatrixCSC(cop_EV)
cop_FE_d = CuSparseMatrixCSC(cop_FE)

12×24 CuSparseMatrixCSC{Int8, Int32} with 48 stored entries:
⢍⢖⠉⠒⠤⣀⠀⠀⠀⠀⠀⠀
⠀⠀⠑⠑⠑⠑⠤⣀⠤⣀⠀⠀
⠀⠀⠀⠀⠀⠀⠑⠑⢄⢄⢍⢖

In [99]:
[W_d, cop_EV_d, cop_FE_d]

2-element Vector{CuSparseMatrixCSC{Tv, Int32} where Tv}:
 sparse(Int32[1, 2, 3, 4, 5, 6, 7, 8, 13, 14  …  3, 4, 5, 6, 7, 8, 10, 12, 14, 16], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [-0.5, -0.5, -0.5, -0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0  …  -0.5, 0.5, -0.5, 0.5, -0.5, 0.5, 1.0, 1.0, 1.0, 1.0], 16, 3)
 sparse(Int32[1, 5, 9, 1, 6, 10, 2, 5, 11, 2  …  21, 15, 20, 22, 16, 19, 23, 16, 20, 24], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  13, 14, 14, 14, 15, 15, 15, 16, 16, 16], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 24, 16)